<a href="https://colab.research.google.com/github/andrewbowen19/pythonBridgeCUNY/blob/main/AndrewBowen_MathWithPython_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Do not use built in programs to do the following.  Write the programs to demonstrate that you understand the math behind them.  Provide all solutions in an .html file. 
1.  Write a Python program to compute the eigenvalues and right eigenvectors of a given square array.
2.  Write a Python program to compute the factor of a given array by Singular Value Decomposition
3.  Write a Python program to compute the determinant of an array.

### Eigenvalues/eigenvectors

The eigenvalues $\lambda_n$ of a square matrix A are found when the characteristic equation $det(A - \lambda I) = 0 $. Going to use the `determinant` function defined below to calculate the eigenvalues

In [7]:
import numpy as np

# Sample matrix -- randomly generated
np.random.seed(1234)

my_matrix = np.random.randint(0, 10, (4,4))
small_matrix = np.array([[1,1], [0, 3]]) ## Sample 'small' matrix with known eigenvals 1 & 3 -- testing purposes

my_matrix

array([[3, 6, 5, 4],
       [8, 9, 1, 7],
       [9, 6, 8, 0],
       [5, 0, 9, 6]])

## Eigenvalues/Eigenvectors

Similar to our SVD attempt below, trying to write our own eigenvalue/eigenvector computation algorithm/functions. Using the `np.linalg.eig` function as a safety net.

Outlining our math/logic for the `get_eigenvector` function

In [8]:
from numpy.linalg import LinAlgError
import scipy
print('Numpy soln:')
sample_matrix = np.array([[-2, -2, 4], [-4, 1, 2],[2, 2, 5]], float)
print(np.linalg.eig(sample_matrix))
print("------------------------")

def get_eigenvector(matrix: np.array, l: float):
  '''Calculate eigenvector for a given matrix and eigenvalue'''
  n = matrix.shape[0]  # Since we're dealing with square matrices, we can assume n = m
  lambda_matrix = np.zeros(matrix.shape, float)
  np.fill_diagonal(lambda_matrix, l)
  print(f'Lambda matrix: {lambda_matrix}')

  # Subtracting eigenvalue across diagonal
  a = matrix - lambda_matrix
  print(a)
  # Trying each basis vector in n-dimensional space
  b = np.zeros((n, 1))

  eigenvector = np.linalg.solve(a, b)

  print(eigenvector)

  return eigenvector

# Running into a bug where we are returning either the zero vector or eigenvectors with huge component vals
# Coming from call of np.linalg.solve --
v = get_eigenvector(sample_matrix, 3)


Numpy soln:
(array([-5.,  3.,  6.]), array([[ 7.57033299e-01,  5.34522484e-01,  4.47213595e-01],
       [ 5.94811877e-01, -8.01783726e-01,  4.78602940e-17],
       [-2.70369035e-01,  2.67261242e-01,  8.94427191e-01]]))
------------------------
Lambda matrix: [[3. 0. 0.]
 [0. 3. 0.]
 [0. 0. 3.]]
[[-5. -2.  4.]
 [-4. -2.  2.]
 [ 2.  2.  2.]]
[[-0.]
 [ 0.]
 [-0.]]


In [9]:
import sympy
from sympy.solvers import solve

def construct_identity_matrix(matrix:list) -> list:
  '''Construct identity matrix of same shape of a given matrix
  Could just use np.identity for simplicity
  '''
  n = len(matrix)
  identity_matrix = [[0 for col in matrix[0]] for row in matrix]
  # Setting diagonal entries to one
  for i in range(n):
    for j in range(n):
      if i == j:
        identity_matrix[i][j] = 1
      else:
        identity_matrix[i][j] = 0

  return identity_matrix

def get_eigenvalues(matrix:list):
  """Compute eigenvalues and eigenvectors of a given n x n matrix/array"""
  n = len(matrix)
  # Converting from list to array for easier matrix multiplication/addition easier via np
  identity_matrix = np.array(construct_identity_matrix(matrix))

  eigenval =  sympy.Symbol("lambda")
  I = identity_matrix * eigenval
  char_matrix = matrix - I
  print(char_matrix)

  # Need to determine where the det of our char_matrix is 0
  # Using sympy.solve to find solns to characteristic eq
  char_eq = sympy.Matrix(char_matrix).det()

  print("Characteristic Eq.:", char_eq)
  eigenvalues = solve(char_eq, eigenval)
  
  print(type(eigenvalues), eigenvalues, len(eigenvalues))

  # Calculating eigenvectors using our get_eigenvector function
  eigenvectors = []
  for e in eigenvalues:
    print(f"Computing eigenvector for eigenvalue: {e}")

    # Running into LinAlgErrors when I try to run
    try:
      eigenvector = get_eigenvector(matrix, e)
      eigenvectors.append(eigenvector)
    except LinAlgError:
      eigenvectors.append(np.zeros((n, 1)))

  return eigenvalues, eigenvectors


evals = get_eigenvalues(sample_matrix)

# evals
# a simpler way would be to use the numpy.linalg.eig function (used below)

[[-lambda - 2.0 -2.00000000000000 4.00000000000000]
 [-4.00000000000000 1.0 - lambda 2.00000000000000]
 [2.00000000000000 2.00000000000000 5.0 - lambda]]
Characteristic Eq.: -lambda**3 + 4.0*lambda**2 + 27.0*lambda - 90.0
<class 'list'> [-5.00000000000000, 3.00000000000000, 6.00000000000000] 3
Computing eigenvector for eigenvalue: -5.00000000000000
Lambda matrix: [[-5.  0.  0.]
 [ 0. -5.  0.]
 [ 0.  0. -5.]]
[[ 3. -2.  4.]
 [-4.  6.  2.]
 [ 2.  2. 10.]]
Computing eigenvector for eigenvalue: 3.00000000000000
Lambda matrix: [[3. 0. 0.]
 [0. 3. 0.]
 [0. 0. 3.]]
[[-5. -2.  4.]
 [-4. -2.  2.]
 [ 2.  2.  2.]]
[[-0.]
 [ 0.]
 [-0.]]
Computing eigenvector for eigenvalue: 6.00000000000000
Lambda matrix: [[6. 0. 0.]
 [0. 6. 0.]
 [0. 0. 6.]]
[[-8. -2.  4.]
 [-4. -5.  2.]
 [ 2.  2. -1.]]


In [10]:
# using numpy.linalg.eig function to calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(my_matrix)

print(f"Eigenvalues: {eigenvalues}")
print(f"Eigenvectors: {eigenvectors}")


# Generalize to a function for any matrix
def get_eig(matrix):
  '''
  Returns eigenvalues and eigenvectors of a given array/matrix

  parameters:
    matrix (np.array): n x n matrix supplied as a numpy array type
  '''
  return np.linalg.eig(matrix)


# Initialize sample random matrix
get_eig(my_matrix)

Eigenvalues: [21.49583862+0.j         -3.192808  +0.j          3.84848469+6.07810701j
  3.84848469-6.07810701j]
Eigenvectors: [[ 0.42769167+0.j          0.79462986+0.j         -0.0399256 -0.02143891j
  -0.0399256 +0.02143891j]
 [ 0.5693551 +0.j         -0.46127325+0.j         -0.3204601 -0.43631538j
  -0.3204601 +0.43631538j]
 [ 0.53834043+0.j         -0.3916827 +0.j         -0.14047031+0.47140593j
  -0.14047031-0.47140593j]
 [ 0.45067082+0.j         -0.04873429+0.j          0.68038598+0.j
   0.68038598-0.j        ]]


(array([21.49583862+0.j        , -3.192808  +0.j        ,
         3.84848469+6.07810701j,  3.84848469-6.07810701j]),
 array([[ 0.42769167+0.j        ,  0.79462986+0.j        ,
         -0.0399256 -0.02143891j, -0.0399256 +0.02143891j],
        [ 0.5693551 +0.j        , -0.46127325+0.j        ,
         -0.3204601 -0.43631538j, -0.3204601 +0.43631538j],
        [ 0.53834043+0.j        , -0.3916827 +0.j        ,
         -0.14047031+0.47140593j, -0.14047031-0.47140593j],
        [ 0.45067082+0.j        , -0.04873429+0.j        ,
          0.68038598+0.j        ,  0.68038598-0.j        ]]))

## Single-Value Decomposition

Trying to write our own SVD algortithm before defaulting to the one provided from `scipy.linalg`.

We use the formula provided in the reading:

$A = U ⋅ \Sigma \cdot V^T$

[We know](https://en.wikipedia.org/wiki/Singular_value_decomposition#Relation_to_eigenvalue_decomposition) the following: 
- The columns of $V$ are eigenvectors of $A^TA$.
- The columns of U (left-singular values) are the eignevectors of $AA^T$.
- The non-zero elements of 
$\Sigma$ (singular values matrix) are the square roots of the non-zero eigenvalues of $A^TA$ or $AA^T$.

Citing wikipedia here but I have found [other sources](https://web.mit.edu/be.400/www/SVD/Singular_Value_Decomposition.htm) that list this condition. I'm going to use the `np.linalg.eig` function to solve for eigenvectors/eigenvalues of $A^TA$ and $AA^T$here because my eigenvector function has a bug currently.



In [11]:
def singularValueDecomposition(A: np.array):
  '''Perform singular value decomposition on a given array/matrx.
  Return U, sigma, Vt matrices
  '''
  # Computing Vt -- right-singular values of A
  # (row eigenvectors of AtA) -- if we transpose it to just get V we'll get the column eigenvector
  AtA = A.transpose() @ A
  evals_V, Vt = np.linalg.eig(AtA)
  Vt = Vt.transpose()

  # Computing left-singular values of A
  AAt = A @ A.transpose()
  evals_U, U = np.linalg.eig(AAt)

  # Computing sigma matrix -- singular values of A (along diagonal)
  print(evals_U, evals_V)
  sigma_U = np.sqrt(evals_U)
  print(f'Sigma (U): {sigma_U}')

  sigma = np.diag(sigma_U)

  return U, sigma, Vt

svd_matrix = np.array([[2, 4], [1, 3], [0, 0], [0, 0]])
U, S, Vt = singularValueDecomposition(svd_matrix)

print(f"U: {U}\n")
print(f"Sigma: {S}")
print(f"V-tanspose: {Vt}")



[29.86606875  0.13393125  0.          0.        ] [ 0.13393125 29.86606875]
Sigma (U): [5.4649857  0.36596619 0.         0.        ]
U: [[ 0.81741556 -0.57604844  0.          0.        ]
 [ 0.57604844  0.81741556  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]

Sigma: [[5.4649857  0.         0.         0.        ]
 [0.         0.36596619 0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
V-tanspose: [[-0.9145143   0.40455358]
 [-0.40455358 -0.9145143 ]]


In [12]:
# Using svd function from scipy.linalg package
from scipy.linalg import svd

def compute_svd(A):
  '''
  Compute Single-Value Decomposition for a given n x m matrix

  parameters:
    arr (np.array): n x n matrix supplied as a numpy array type

  returns:
    U (np.array): left-singluar vectors of A
    sigma (np.array): singular vectors of A
    Vt (np.array): right-singular vectors of A
  '''
  return svd(A)

U, sigma, Vt = compute_svd(svd_matrix)

# Printing out Vt's transpose so we have the right values as column vectors
# They look the same (Vt transposed but same row/col vector values) as our singularValueDecomposition function above
print(f"left-singular values (U): {U}")
print(f"Singular values (Sigma): {sigma}")
print(f"Right-singular values: {Vt}")

left-singular values (U): [[-0.81741556 -0.57604844  0.          0.        ]
 [-0.57604844  0.81741556  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
Singular values (Sigma): [5.4649857  0.36596619]
Right-singular values: [[-0.40455358 -0.9145143 ]
 [-0.9145143   0.40455358]]


## Determinant of an array

Implementing the formula from [this vid](https://www.youtube.com/watch?v=Dp5ShwFABUo):


> $det(A) = \sum_{j=1}^{n} (-1)^{i + j}a_{ij}det(A_{ij})$

Keeping $i$ fixed at the starting row (indexed at 0 in python), and looping over the columns to generate the sub-matrices

I'm using the [python `copy` library](https://docs.python.org/3/library/copy.html) so the sub-matrix generation doesn't affect the original matrix object.

In [16]:
from copy import deepcopy

def get_sub_matrix(matrix:list, row:int, column:int) -> list:
  '''Removes given row and column values from a matrix
  Will use when determining sub-matricies we need to find the determinant of
  '''
  # Making a. copy of the matrix arg so the original matrix isn't affected
  new_matrix = deepcopy(matrix)

  # Removing the selected row and column vals from other rows in matrix
  new_matrix.remove(matrix[row])
  for i in range(len(new_matrix)):
    new_matrix[i].pop(column)

  return new_matrix 


def determinant(matrix:list) -> int:
  """Compute determinant of a square matrix"""
  n = len(matrix)
  
  det = 1
  # Going to keep indexing zero-based to be consistent with python
  # Increment our determinant sum recursively
  for j in range(0, n):
    sub_matrix = get_sub_matrix(matrix, 0, j)
    smaller_det = ((-1)**(j)) * matrix[0][j] * determinant(sub_matrix)
    det += smaller_det

  return det


d = determinant(my_matrix.tolist())

print(f"Determinant: {d}")


Determinant: -3750


In [14]:
# Alternatively, using the numpy.linalg.det function is a lot easier :)
d = np.linalg.det(my_matrix)

print(f"Determinant: {d}")

Determinant: -3552.000000000004
